# Lecture 22
## Tuesday, November 15th, 2022
### SQL Database Exercise (I)

# `SQLite` Exercises

Today you will work with the `candidates.txt` and `contributors.txt` datasets to
create a database in `python` using `SQLite`. This is a hands-on lecture.
You may do these tasks in a Jupyter notebook.

The exercises will consist of a sequence of steps to help illustrate basic
commands.

<a id='deliverables'></a>
# Exercise Deliverables

> 1. Copy the Jupyter notebook along with `candidates.txt` and
>    `contributors.txt` into `lab/pp12` in your private Git repository and
>    commit on your default branch.
> 2. For each step in the exercise notebook, there are instructions labeled
>    "**Do the following:**" (except for Setup and Interlude). Put all the code
>    for those instructions in _code cell(s) immediately following the
>    instructions_. The code in those cells should be regular Python code.
>    You should place comments where appropriate that describe your intentions.
>    **Note:** To get the
>    `pandas` tables to display in a cell, use `display()`.
> 3. Save and close your database.  Be sure to upload your database in
>    `lab/pp12` as well.  Please name your database **`lecture22.sqlite`**.

## Table of Contents

[Setup](#setup)

[Interlude](#interlude): Not required but highly recommended.

[Step 1](#step_1)

[Step 2](#step_2)

[Step 3](#step_3)

[Step 4](#step_4)

[Step 5](#step_5)

[Step 6](#step_6)

[Step 7](#step_7)

[Step 8](#step_8)


<a id='setup'></a>
# Setup

You should import `sqlite3` again like in the lecture.

In [ ]:
import sqlite3

We will also use a basic `pandas` feature to display tables in the database.

In [ ]:
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

from IPython.display import display

Now we create the tables in the database (similar to lecture).

In [ ]:
db = sqlite3.connect('lecture22.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

Next we load the data for the candidates:

In [ ]:
with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_name, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)
db.commit()

<a id='interlude'></a>
## Interlude

Now that you have values in the tables of the database, it would be convenient
to be able to visualize those tables in some way.  We'll write a little helper
function to accomplish this.

In [ ]:
def viz_tables(query, *, database=db):
    return pd.read_sql_query(query, database)

Here's how we can use our helper function.  It gives a pretty nice visualization
of our table.

In [ ]:
viz_tables('''SELECT * FROM candidates''')

<a id='step_1'></a>
# Step 1

We still need to load the data in the contributors table.

### Do the following:

* Load the data in the `contributors` table by loading the data from the
  `contributors.txt` file.  **You are not allowed to use a `for`-loop in this
  task (comprehensions are allowed)**.
* Display the contributors table.

In [ ]:
with open ("contributors.txt") as contributors:
    next(contributors)  # skip header line
    cursor.executemany(
        '''INSERT INTO contributors
           (last_name, first_name, middle_name, street_1, street_2, city,
           state, zip, amount, date, candidate_id)
           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
        [row.strip().split('|')[1:] for row in contributors.readlines()])
# Visualize the table
display(viz_tables('''SELECT * FROM contributors'''))

<a id='step_2'></a>
# Step 2:  Various Queries

We can query our database for entries with certain characteristics.  For
example, we can query the `candidates` table for entries whose middle name
fields are not empty.

In [ ]:
query = '''SELECT * FROM candidates WHERE middle_name <> ""'''
viz_tables(query)

We can also see how many entries satisfy the query:

In [ ]:
print("{} candidates have a middle initial.".format(viz_tables(query).shape[0]))

This used the `shape` method on the returned `pandas` table. You'll get to
practice counting in SQL later.

### Do the following queries:

* Display the contributors where the state is "PA"
* Display the contributors where the amount contributed is greater than
  $\$1000.00$.
* Display the contributors from the state "UT" where the amount contributed is
  greater than $\$1000.00$.
* Display the contributors who didn't list their state
  - **Hint**:  Match `state` to the empty string
* Display the contributors from "WA" or "PA"
* Display the contributors who contributed between $\$100.00$ and $\$200.00$.
  - **Hint**: You can use the `BETWEEN 100.00 and 200.00` clause.

In [ ]:
# State PA
display(viz_tables('''SELECT * FROM contributors WHERE UPPER(state) = "PA"'''))
# More than 1000.0
display(viz_tables('''SELECT * FROM contributors WHERE amount > 1000.0'''))
# More than 1000.0
display(viz_tables('''SELECT * FROM contributors WHERE UPPER(state) = "UT" AND amount > 1000.0'''))
# Unknown state
display(viz_tables('''SELECT * FROM contributors WHERE state = ""'''))
# Contributors from two states
# Alternative: display(viz_tables('''SELECT * FROM contributors WHERE UPPER(state) IN ("WA", "PA")'''))
display(viz_tables('''SELECT * FROM contributors WHERE UPPER(state) = "WA" OR UPPER(state) = "PA"'''))
# Contributions between 100.0 and 200.0
display(viz_tables('''SELECT * FROM contributors WHERE amount BETWEEN 100.0 AND 200.0'''))

<a id='step_3'></a>
# Step 3:  Sorting

It could be beneficial to sort by one of the attributes in the database.  The
following cell contains a basic sorting demo. Run it and try to understand what
happened.

In [ ]:
viz_tables('''SELECT * FROM candidates ORDER BY id DESC''')

### Do the following sorts on the specified tables:

* Sort the `candidates` table by `last_name`.
* Sort the `contributors` table by the `amount` in descending order where `amount`
  is restricted to be between $\$1000.00$ and $\$5000.00$.
  - **Hint:** In your SQL command, start with getting the amount between the
    specified range followed by the sort. This will all be done in one line.
* Sort the contributors who donated between $\$1000.00$ and $\$5000.00$ by
  `candidate_id` and then by `amount` in descending order.
  - **Hint**:  Multiple orderings can be accomplished by separating requests
    after `ORDER BY` with commas.
  - e.g. `ORDER BY amount ASC, last_name DESC`

In [ ]:
# Sort candidates by last name
display(viz_tables('''SELECT * FROM candidates ORDER BY last_name'''))
# Sort by amount in decending order
display(viz_tables('''SELECT * FROM contributors WHERE amount BETWEEN 1000.0 AND 5000.0 ORDER BY amount DESC'''))
# Sort by amount in range and order by candidate_id and then by amount
display(viz_tables('''SELECT * FROM contributors WHERE amount BETWEEN 1000.0 AND 5000.0 ORDER BY candidate_id, amount DESC'''))

<a id='step_4'></a>
# Step 4:  Selecting Columns

So far, we've been selecting all columns from a table (i.e. `SELECT * FROM`).
Often, we just want to select specific columns (e.g. `SELECT amount FROM`).

In [ ]:
viz_tables('''SELECT last_name, party FROM candidates''')

Using the `DISTINCT` clause, you remove duplicate rows.

In [ ]:
viz_tables('''SELECT DISTINCT party FROM candidates''')

### Do the following:

* Get the first and last name of contributors.  Make sure each row has distinct
  values.

In [ ]:
display(viz_tables('''SELECT DISTINCT last_name, first_name FROM contributors'''))

<a id='step_5'></a>
# Step 5:  Altering Tables

The `ALTER` clause allows us to modify tables in our database.  Here, we add a
new column to our candidates table called `full_name`.

In [ ]:
cursor.execute('''ALTER TABLE candidates ADD COLUMN full_name TEXT''')
viz_tables('''SELECT * FROM candidates''')

What if we want to rename or delete a column?  It can't be done with `SQLite`
with a single command.  We need to follow some roundabout steps (see [`SQLite`
ALTER TABLE](http://www.sqlitetutorial.net/sqlite-alter-table/)).  We won't
consider this case at the moment.

For now, let's put a few commands together to populate the `full_name` column.

In [ ]:
query = '''SELECT id, last_name, first_name FROM candidates''' # Select a few columns
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)

update = '''UPDATE candidates SET full_name = ? WHERE id = ?''' # Update the table
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM candidates'''
viz_tables(query)

Here's another update, this time on an existing column.

In [ ]:
update = '''UPDATE candidates SET full_name = "WINNER" WHERE last_name = "Obama"'''
cursor.execute(update)
update = '''UPDATE candidates SET full_name = "RUNNER-UP" WHERE last_name = "McCain"'''
cursor.execute(update)
viz_tables(query)

### Do the following:

* Add a new column to the contributors table called `full_name`.  The value in
  that column should be in the form `last_name, first_name`.
* Change the value in the `full_name` column to the string `"Too Much"` if
  someone donated more than $\$1000.00$.

In [ ]:
# Add full_name column
cursor.execute('''ALTER TABLE contributors ADD COLUMN full_name TEXT''')
query = '''SELECT id, last_name, first_name FROM contributors'''
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0])
                    for attr in cursor.execute(query).fetchall()]
update = '''UPDATE contributors SET full_name = ? WHERE id = ?'''
cursor.executemany(update, full_name_and_id)
# Change full_name value to "Too Much"
update = '''UPDATE contributors SET full_name = "Too Much" WHERE amount > 1000.0'''
cursor.execute(update)
display(viz_tables('''SELECT * FROM contributors'''))

<a id='step_6'></a>
# Step 6: Aggregation

You can perform reduction operations on the values in the database.  For
example, you can compute the maximum, minimum, sum or the total number from
multiple input values. Here's a little example:

In [ ]:
viz_tables('''SELECT *, MAX(amount) AS max_amount FROM contributors''')

### Do the following:

* Modify the demo to only output the max amount.
    * **Hints:**
        - Instead of using `SELECT *, MAX(amount) ...` you can try `SELECT
          MAX(amount) ...`
        - You will want to use `cursor.execute()` here and then `fetchall()`.
        - Do not display your results in a table. It is sufficient to write
          the answer out to the screen as a single number.
* Count how many donations there were above $\$1000.00$.
    * **Hint:** There is a `COUNT` function.
* Calculate the average *positive* donation.
    * **Hint:** There is an `AVG` function.
* Calculate the average contribution from each state and display in a table.
  Restrict to positive values again.
  - **Hint**:  Use code that looks like: `"SELECT state,SUM(amount) FROM
    contributors GROUP BY state"`.

In [ ]:
# How many donations about $1000.00
how_many = cursor.execute(
    "SELECT COUNT(amount) AS num_above FROM contributors WHERE amount > 1000.0"
)
print(how_many.fetchall())
# Average donation
avg_don = cursor.execute(
    "SELECT AVG(amount) FROM contributors WHERE amount > 0.0")
print(avg_don.fetchall())
# Average contribution from each state
display(viz_tables('''SELECT state, AVG(amount) FROM contributors GROUP BY state'''))

<a id='step_7'></a>
# Step 7: DELETE

We have already noted that `SQLite` can't drop columns in a straightforward
manner.  However, it can delete rows quite simply.  Here's the syntax:

In [ ]:
deletion = '''DELETE FROM table_name WHERE condition'''

### Do the following:

* Delete rows in the `contributors` table with last name "Ahrens".

In [ ]:
# Delete a row
deletion = '''DELETE FROM contributors WHERE last_name = "Ahrens"'''
cursor.execute(deletion)
display(viz_tables('''SELECT * FROM contributors'''))

<a id='step_8'></a>
# Step 8:  LIMIT

The `LIMIT` clause offers convenient functionality.  It allows you to constrain
the number of rows returned by your query.  It shows up in many guises.

In [ ]:
viz_tables('''SELECT * FROM candidates LIMIT 3''')

In [ ]:
viz_tables('''SELECT * FROM candidates LIMIT 4 OFFSET 5''')

In [ ]:
viz_tables('''SELECT * FROM candidates ORDER BY last_name LIMIT 4 OFFSET 5''')

### Do the following:

* Query and display the ten most generous donors.
* Query and display the ten least generous donors who donated a positive amount
  of money (since the data we have has some negative numbers in it...).

In [ ]:
# 10 most generous donors
display(viz_tables('''SELECT * FROM contributors ORDER BY amount DESC LIMIT 10'''))
# 10 least generous donors
display(viz_tables('''SELECT * FROM contributors WHERE amount > 0 ORDER BY amount LIMIT 10'''))

# Save

Don't forget to save all of these changes to your database using `db.commit()`.
Before closing your editor or IDE, be sure to close the database connection with
`db.close()`.

In [ ]:
db.commit()
db.close()